In [1]:
import pandas as pd
import pyarrow as pa
from collections import defaultdict
from pyarrow import parquet as pq
import gc

In [2]:
train = pd.read_csv('../Data/Train/sales_train_validation.csv')
train = train.drop(['item_id','dept_id','cat_id','store_id','state_id'],axis =1)
train.index = train.id
train = train.drop('id',axis = 1)

In [20]:
def get_time_tensor(arr,window_size, start_date=0):
    Xtensor = []
    Ytensor = []
    date_interval = []
    for ix, v in enumerate(arr):
        if ix+1 >= window_size and ix+1 < len(arr):
            tmp = arr[ix+1-window_size:ix+1]
            Xtensor.append(tmp)
            Ytensor.append(arr[ix+1])
            date_interval.append(list(range(ix+1-window_size+start_date,ix+1+start_date)))
    return Xtensor,Ytensor,date_interval

def _get_item_id(fullIndex):
    tmp = fullIndex.split('_')
    return '_'.join(tmp[:3])

def _get_store_id(fullIndex):
    tmp = fullIndex.split('_')
    return '_'.join(tmp[3:5])

def _get_cate_info(fullIndex,cateInfoDir):
    item_id = _get_item_id(fullIndex)
    store_id = _get_store_id(fullIndex)
    return pd.read_parquet(cateInfoDir,filters = [("item_id",'=',str(item_id)),("store_id",'=',str(store_id))])

def _get_events(calendar_dir,sdate,edate):
    df = pd.read_csv(calendar_dir)
    df['d_num'] = df.d.apply(lambda x: x.replace('d_','')).astype('int')
    return df[df.d_num.apply(lambda x: x <= edate and x >= sdate)]

def _get_future_events(calendar_dir, sdate,duration):
    edate = sdate+duration-1
    return _get_events(calendar_dir,sdate,edate)


In [22]:
_get_future_events('../Data/Train/calendar.csv',100,10)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,d_num
99,2011-05-08,11115,Sunday,2,5,2011,d_100,Mother's day,Cultural,NaN,NaN,1,0,1,100
100,2011-05-09,11115,Monday,3,5,2011,d_101,NaN,NaN,NaN,NaN,1,1,1,101
101,2011-05-10,11115,Tuesday,4,5,2011,d_102,NaN,NaN,NaN,NaN,1,0,0,102
102,2011-05-11,11115,Wednesday,5,5,2011,d_103,NaN,NaN,NaN,NaN,0,1,1,103
103,2011-05-12,11115,Thursday,6,5,2011,d_104,NaN,NaN,NaN,NaN,0,1,1,104
104,2011-05-13,11115,Friday,7,5,2011,d_105,NaN,NaN,NaN,NaN,0,1,0,105
105,2011-05-14,11116,Saturday,1,5,2011,d_106,NaN,NaN,NaN,NaN,0,0,1,106
106,2011-05-15,11116,Sunday,2,5,2011,d_107,NaN,NaN,NaN,NaN,0,1,1,107
107,2011-05-16,11116,Monday,3,5,2011,d_108,NaN,NaN,NaN,NaN,0,0,0,108
108,2011-05-17,11116,Tuesday,4,5,2011,d_109,NaN,NaN,NaN,NaN,0,0,0,109


In [36]:
df = pd.read_csv('../Data/Train/sell_prices.csv')

In [43]:
b = df.groupby(['store_id','item_id']).wm_yr_wk.max()

In [45]:
b[b != 11621]

Series([], Name: wm_yr_wk, dtype: int64)

In [32]:
1969-1913

56